In [1]:
from pykml import parser
import folium
from shapely.geometry import LineString, Point, Polygon, shape
from pyproj import Transformer
import webbrowser
import geojson

INPUT & OUTPUT

In [37]:
###Input###
kml_file = 'Jaktim - 3.kml'

###Output###
GeoJSON_file = 'Jaktim - 3.geojson'
HTML_file = 'Polygon.html'

# buffer distance (meters)
buffer_distance = 10

# tolerance 
tolerance = 0.00005

KML

In [38]:
with open(kml_file, mode='r', encoding='utf-8') as f:
    root = parser.parse(f).getroot()

coords = root.Document.Placemark.LineString.coordinates.text
coordinates = coords.strip().split() 

#print(coords)

polyline_points = [] 
for coordinate in coordinates:
    lon, lat, __ = coordinate.split(',')
    latitude = float(lat)
    longitude = float(lon)
    polyline_points.append((latitude, longitude))

#print(polyline_points)

BUFFER & POLYGON

In [39]:
in_proj = "EPSG:4326"
out_proj = "EPSG:3857"
transformer = Transformer.from_crs(in_proj, out_proj)
projected_points = [transformer.transform(lat, lon) for lat, lon in polyline_points]

line = LineString(projected_points)
buffered_line = line.buffer(buffer_distance)
buffer_polygon = buffered_line
buffer_polygon_exterior = buffer_polygon.exterior
buffer_polygon_coords = list(buffer_polygon_exterior.coords)
buffer_polygon_latlon = [transformer.transform(x, y, direction='INVERSE') for x, y in buffer_polygon_coords]

poly = Polygon(buffer_polygon_latlon)
simplified_poly = poly.simplify(tolerance, preserve_topology=True)
simplified_coords = list(simplified_poly.exterior.coords)

GeoJSON

In [40]:
geojson_coords = [(lon, lat) for lat, lon in simplified_coords]
polygon = geojson.Feature(geometry=geojson.Polygon([geojson_coords]), properties={})

with open(GeoJSON_file, 'w') as f:
    geojson.dump(polygon, f)

parsing = {
    "type": polygon.geometry.type,
    "coordinates": polygon.geometry.coordinates
}
polygon_area = geojson.dumps(parsing)
print(polygon_area)

{"type": "Polygon", "coordinates": [[[106.901315, -6.245035], [106.904227, -6.246155], [106.904878, -6.246275], [106.909973, -6.247747], [106.910378, -6.24796], [106.910202, -6.248739], [106.911288, -6.248621], [106.913552, -6.24902], [106.917289, -6.248836], [106.917629, -6.25056], [106.917639, -6.251179], [106.917541, -6.251259], [106.917462, -6.251187], [106.917411, -6.250221], [106.917185, -6.249013], [106.913548, -6.249199], [106.911156, -6.248789], [106.910059, -6.248937], [106.909991, -6.248836], [106.910184, -6.248033], [106.908614, -6.247515], [106.906593, -6.247005], [106.903548, -6.246094], [106.901162, -6.245165], [106.901122, -6.245073], [106.901245, -6.244672], [106.901382, -6.244565], [106.901741, -6.244503], [106.902974, -6.245037], [106.904362, -6.245466], [106.905028, -6.245856], [106.906207, -6.246245], [106.907641, -6.246643], [106.909131, -6.246933], [106.912647, -6.247882], [106.913905, -6.248091], [106.915515, -6.248061], [106.920544, -6.247621], [106.922357, -6.

HTML

In [41]:
m = folium.Map(location=polyline_points[0], zoom_start=12)
folium.Polygon(locations=buffer_polygon_latlon, color='red', fill=True, fill_color='red', fill_opacity=0.5).add_to(m)
folium.PolyLine(locations=polyline_points, color='red').add_to(m)
folium.GeoJson(polygon, name="geojson").add_to(m)

m.save(HTML_file)
#webbrowser.open(HTML_file,1)
m